In [6]:
# -*- coding: utf-8 -*-
"""
Created on Fri Oct 26 13:10:46 2018

@author: Aria
"""
import pandas as pd
import numpy as np
import tensorflow as tf

tf.reset_default_graph()

#load Data
train_data = pd.read_csv("pair_data.csv")
train_label = pd.read_csv("pair_label.csv")
test_data = pd.read_csv("test1.csv")
x_train = train_data.iloc[:,:].values
x_train = x_train.astype(np.float)
test = test_data.iloc[:,:].values
test = test.astype(np.float)
#5 abstract dataset labels
train_labels_flat = train_label.iloc[:,0].values.ravel()
#labels_flat = train[[0]].values.ravel()
train_labels_count = np.unique(train_labels_flat).shape[0]
# One_Hot
def dense_to_one_hot(labels_dense,num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels)*num_classes
    labels_one_hot = np.zeros((num_labels,num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()-1]= 1
    return labels_one_hot

#6 eval one_hot
y_train = dense_to_one_hot(train_labels_flat,train_labels_count)
y_train = y_train.astype(np.uint8)

#==============================================================
#打乱顺序
train_data_shape = x_train.shape[0]
test_shape = test.shape[0]
arr = np.arange(train_data_shape)
np.random.shuffle(arr)
x_train = x_train[arr]
y_train = y_train[arr]
#=============================================================
#划分训练集和验证集20%
k=80400
validdata=x_train[:k]
validlabel=y_train[:k]

traindata=x_train[k:]
trainlabel=y_train[k:]
#print(traindata.shape[0])
#print(validdata.shape[0])
#=============================================================
#n_batch = train_data_shape*0.8/100
batch_size = 100

x = tf.placeholder(tf.float32,[None,400])
y = tf.placeholder(tf.float64,[None,10])

#10 def funcation
def weight_variable(shape):
    # inital Weight
    initial = tf.truncated_normal(shape,stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    # initial bias
    initial = tf.constant(0.1,shape = shape)
    return tf.Variable(initial)

def conv2d(x,W):
    # 2D con
    return tf.nn.conv2d(x,W,strides = [1,1,1,1],padding = 'VALID')


#11 make inputdata to 4d tensor
x_image = tf.reshape(x,[-1,2,200,1])

#12 con1
W_conv1 = weight_variable([1,9,1,10])
b_conv1 = bias_variable([10])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)


#12 con2
W_conv2 = weight_variable([2,1,10,10])
b_conv2 = bias_variable([10])
h_conv2 = tf.nn.relu(conv2d(h_conv1,W_conv2)+b_conv2)

#12 con3
W_conv3 = weight_variable([1,3,10,10])
b_conv3 = bias_variable([10])
conv3=tf.nn.conv2d(h_conv2,W_conv3,strides = [1,1,1,1],padding = 'SAME')
h_conv3 = tf.nn.relu(conv3+b_conv3)

h_pool1 = tf.nn.max_pool(h_conv3,ksize=[1,1,3,1],strides=[1,1,3,1],padding='VALID')

#12 con4
W_conv4 = weight_variable([1,3,10,20])
b_conv4 = bias_variable([20])
h_conv4 = tf.nn.relu(conv2d(h_pool1,W_conv4)+b_conv4)

#12 con5
W_conv5 = weight_variable([1,3,20,20])
b_conv5 = bias_variable([20])
h_conv5 = tf.nn.relu(conv2d(h_conv4,W_conv5)+b_conv5)

h_pool2 = tf.nn.max_pool(h_conv5,ksize=[1,1,2,1],strides=[1,1,2,1],padding='VALID')

#12 con6
W_conv6 = weight_variable([1,3,20,40])
b_conv6 = bias_variable([40])
h_conv6 = tf.nn.relu(conv2d(h_pool2,W_conv6)+b_conv6)

#12 con7
W_conv7 = weight_variable([1,3,40,40])
b_conv7 = bias_variable([40])
h_conv7 = tf.nn.relu(conv2d(h_conv6,W_conv7)+b_conv7)

h_pool3 = tf.nn.max_pool(h_conv7,ksize=[1,1,2,1],strides=[1,1,2,1],padding='VALID')

#12 con8
W_conv8 = weight_variable([1,13,40,80])
b_conv8 = bias_variable([80])
h_conv8 = tf.nn.relu(conv2d(h_pool3,W_conv8)+b_conv8)

#12 con9
W_conv9 = weight_variable([1,1,80,80])
b_conv9 = bias_variable([80])
h_conv9 = tf.nn.relu(conv2d(h_conv8,W_conv9)+b_conv9)

#12 con10
W_conv10 = weight_variable([1,1,80,10])
b_conv10 = bias_variable([10])
y_conv = conv2d(h_conv9,W_conv10)+b_conv10

y_out_soft=tf.nn.softmax(y_conv)

y_out=tf.reshape(y_out_soft,[100,10])
out = tf.argmax(y_out,1)

# loss funcation
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits = y_conv))

# gradient descent
train_step_1 = tf.train.AdadeltaOptimizer(learning_rate = 0.001).minimize(loss)

# cal accuracy
correct_prediction= tf.equal(tf.argmax(y,1),tf.argmax(y_out,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
#initial
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# begin train
with tf.Session() as sess:
    ckpt=tf.train.get_checkpoint_state('logs')
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess,ckpt.model_checkpoint_path)
        global_step=ckpt.model_checkpoint_path.split('/')[-1].split('_')[-1]
    for epoch in range(9109,10000):
        for i in range(3210):
            start=(i*batch_size)%321000  #traindata.shape[0]
            end=start+batch_size
            batch_x = traindata[start:end]
            batch_y = trainlabel[start:end]
            #print("di  "+str(i))
            sess.run(train_step_1,feed_dict = {x:batch_x,y:batch_y})
        #cal accuracy
        for i in range(804):
            start=(i*batch_size)%80400  #validdata.shape[0]
            end=start+batch_size
            batch_xx = validdata[start:end]
            batch_yy = validlabel[start:end]
            accuracy_n = sess.run(accuracy,feed_dict = {x:batch_xx,y:batch_yy})
        print("第"+str(epoch)+"轮的准确度是:"+str(accuracy_n))
        #print("第"+str(epoch))
        #global_step.assign(epoch).eval()
        saver.save(sess,"logs/model.ckpt",global_step = epoch)
    
print("6")


writer = tf.summary.FileWriter('Summary/',tf.get_default_graph())
writer.close()

#pred=sess.run(out,feed_dict = {x:test})


INFO:tensorflow:Restoring parameters from logs/model.ckpt-9108
第9109轮的准确度是:0.88
第9110轮的准确度是:0.88
第9111轮的准确度是:0.88
第9112轮的准确度是:0.88
第9113轮的准确度是:0.88
第9114轮的准确度是:0.88
第9115轮的准确度是:0.88
第9116轮的准确度是:0.88
第9117轮的准确度是:0.88
第9118轮的准确度是:0.88
第9119轮的准确度是:0.88
第9120轮的准确度是:0.88
第9121轮的准确度是:0.88
第9122轮的准确度是:0.88
第9123轮的准确度是:0.88
第9124轮的准确度是:0.88
第9125轮的准确度是:0.88
第9126轮的准确度是:0.88
第9127轮的准确度是:0.88
第9128轮的准确度是:0.88
第9129轮的准确度是:0.88
第9130轮的准确度是:0.88
第9131轮的准确度是:0.88
第9132轮的准确度是:0.88
第9133轮的准确度是:0.88
第9134轮的准确度是:0.88
第9135轮的准确度是:0.88
第9136轮的准确度是:0.88
第9137轮的准确度是:0.88
第9138轮的准确度是:0.88
第9139轮的准确度是:0.88
第9140轮的准确度是:0.88
第9141轮的准确度是:0.88
第9142轮的准确度是:0.88
第9143轮的准确度是:0.88
第9144轮的准确度是:0.88
第9145轮的准确度是:0.88
第9146轮的准确度是:0.88
第9147轮的准确度是:0.88
第9148轮的准确度是:0.88
第9149轮的准确度是:0.88
第9150轮的准确度是:0.88
第9151轮的准确度是:0.88
第9152轮的准确度是:0.88
第9153轮的准确度是:0.88
第9154轮的准确度是:0.88
第9155轮的准确度是:0.88
第9156轮的准确度是:0.88
第9157轮的准确度是:0.88
第9158轮的准确度是:0.88
第9159轮的准确度是:0.88
第9160轮的准确度是:0.88
第9161轮的准确度是:0.88
第9162轮的准确度是:0.88
第9163轮的准确度是:0.88
第9

第9588轮的准确度是:0.89
第9589轮的准确度是:0.89
第9590轮的准确度是:0.89
第9591轮的准确度是:0.89
第9592轮的准确度是:0.89
第9593轮的准确度是:0.89
第9594轮的准确度是:0.89
第9595轮的准确度是:0.89
第9596轮的准确度是:0.89
第9597轮的准确度是:0.89
第9598轮的准确度是:0.89
第9599轮的准确度是:0.89
第9600轮的准确度是:0.89
第9601轮的准确度是:0.89
第9602轮的准确度是:0.89
第9603轮的准确度是:0.89
第9604轮的准确度是:0.89
第9605轮的准确度是:0.89
第9606轮的准确度是:0.89
第9607轮的准确度是:0.89
第9608轮的准确度是:0.89
第9609轮的准确度是:0.89
第9610轮的准确度是:0.89
第9611轮的准确度是:0.89
第9612轮的准确度是:0.89
第9613轮的准确度是:0.89
第9614轮的准确度是:0.89
第9615轮的准确度是:0.89
第9616轮的准确度是:0.89
第9617轮的准确度是:0.89
第9618轮的准确度是:0.89
第9619轮的准确度是:0.89
第9620轮的准确度是:0.89
第9621轮的准确度是:0.89
第9622轮的准确度是:0.89
第9623轮的准确度是:0.89
第9624轮的准确度是:0.89
第9625轮的准确度是:0.89
第9626轮的准确度是:0.89
第9627轮的准确度是:0.89
第9628轮的准确度是:0.89
第9629轮的准确度是:0.89
第9630轮的准确度是:0.89
第9631轮的准确度是:0.89
第9632轮的准确度是:0.89
第9633轮的准确度是:0.89
第9634轮的准确度是:0.89
第9635轮的准确度是:0.89
第9636轮的准确度是:0.89
第9637轮的准确度是:0.89
第9638轮的准确度是:0.89
第9639轮的准确度是:0.89
第9640轮的准确度是:0.89
第9641轮的准确度是:0.89
第9642轮的准确度是:0.89
第9643轮的准确度是:0.89
第9644轮的准确度是:0.89
第9645轮的准确度是:0.89
第9646轮的准确度是:0.

KeyboardInterrupt: 

In [22]:
# -*- coding: utf-8 -*-
"""
Created on Fri Oct 26 13:10:46 2018

@author: Aria
"""
import pandas as pd
import numpy as np
import tensorflow as tf

tf.reset_default_graph()

#load Data
train_data = pd.read_csv("pair_data.csv")
train_label = pd.read_csv("pair_label.csv")
test_data = pd.read_csv("test1.csv")
x_train = train_data.iloc[:,:].values
x_train = x_train.astype(np.float)
test = test_data.iloc[:,:].values
test = test.astype(np.float)
#5 abstract dataset labels
train_labels_flat = train_label.iloc[:,0].values.ravel()
#labels_flat = train[[0]].values.ravel()
train_labels_count = np.unique(train_labels_flat).shape[0]
# One_Hot
def dense_to_one_hot(labels_dense,num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels)*num_classes
    labels_one_hot = np.zeros((num_labels,num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()-1]= 1
    return labels_one_hot

#6 eval one_hot
y_train = dense_to_one_hot(train_labels_flat,train_labels_count)
y_train = y_train.astype(np.uint8)

#==============================================================
#打乱顺序
train_data_shape = x_train.shape[0]
test_shape = test.shape[0]
arr = np.arange(train_data_shape)
np.random.shuffle(arr)
x_train = x_train[arr]
y_train = y_train[arr]
#=============================================================
#划分训练集和验证集20%
k=80400
validdata=x_train[:k]
validlabel=y_train[:k]

traindata=x_train[k:]
trainlabel=y_train[k:]
#print(traindata.shape[0])
#print(validdata.shape[0])
#=============================================================
#n_batch = train_data_shape*0.8/100
batch_size = 100

x = tf.placeholder(tf.float32,[None,400])
y = tf.placeholder(tf.float64,[None,10])

#10 def funcation
def weight_variable(shape):
    # inital Weight
    initial = tf.truncated_normal(shape,stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    # initial bias
    initial = tf.constant(0.1,shape = shape)
    return tf.Variable(initial)

def conv2d(x,W):
    # 2D con
    return tf.nn.conv2d(x,W,strides = [1,1,1,1],padding = 'VALID')


#11 make inputdata to 4d tensor
x_image = tf.reshape(x,[-1,2,200,1])

#12 con1
W_conv1 = weight_variable([1,9,1,10])
b_conv1 = bias_variable([10])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)


#12 con2
W_conv2 = weight_variable([2,1,10,10])
b_conv2 = bias_variable([10])
h_conv2 = tf.nn.relu(conv2d(h_conv1,W_conv2)+b_conv2)

#12 con3
W_conv3 = weight_variable([1,3,10,10])
b_conv3 = bias_variable([10])
conv3=tf.nn.conv2d(h_conv2,W_conv3,strides = [1,1,1,1],padding = 'SAME')
h_conv3 = tf.nn.relu(conv3+b_conv3)

h_pool1 = tf.nn.max_pool(h_conv3,ksize=[1,1,3,1],strides=[1,1,3,1],padding='VALID')

#12 con4
W_conv4 = weight_variable([1,3,10,20])
b_conv4 = bias_variable([20])
h_conv4 = tf.nn.relu(conv2d(h_pool1,W_conv4)+b_conv4)

#12 con5
W_conv5 = weight_variable([1,3,20,20])
b_conv5 = bias_variable([20])
h_conv5 = tf.nn.relu(conv2d(h_conv4,W_conv5)+b_conv5)

h_pool2 = tf.nn.max_pool(h_conv5,ksize=[1,1,2,1],strides=[1,1,2,1],padding='VALID')

#12 con6
W_conv6 = weight_variable([1,3,20,40])
b_conv6 = bias_variable([40])
h_conv6 = tf.nn.relu(conv2d(h_pool2,W_conv6)+b_conv6)

#12 con7
W_conv7 = weight_variable([1,3,40,40])
b_conv7 = bias_variable([40])
h_conv7 = tf.nn.relu(conv2d(h_conv6,W_conv7)+b_conv7)

h_pool3 = tf.nn.max_pool(h_conv7,ksize=[1,1,2,1],strides=[1,1,2,1],padding='VALID')

#12 con8
W_conv8 = weight_variable([1,13,40,80])
b_conv8 = bias_variable([80])
h_conv8 = tf.nn.relu(conv2d(h_pool3,W_conv8)+b_conv8)

#12 con9
W_conv9 = weight_variable([1,1,80,80])
b_conv9 = bias_variable([80])
h_conv9 = tf.nn.relu(conv2d(h_conv8,W_conv9)+b_conv9)

#12 con10
W_conv10 = weight_variable([1,1,80,10])
b_conv10 = bias_variable([10])
y_conv = conv2d(h_conv9,W_conv10)+b_conv10

y_out_soft=tf.nn.softmax(y_conv)

y_out=tf.reshape(y_out_soft,[100,10])
out = tf.argmax(y_out,1)

# loss funcation
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits = y_conv))

# gradient descent
train_step_1 = tf.train.AdadeltaOptimizer(learning_rate = 0.001).minimize(loss)

# cal accuracy
correct_prediction= tf.equal(tf.argmax(y,1),tf.argmax(y_out,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
#initial
init = tf.global_variables_initializer()
saver = tf.train.Saver()
pred=[1]
# begin train
with tf.Session() as sess:
    ckpt=tf.train.get_checkpoint_state('logs')
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess,ckpt.model_checkpoint_path)
        global_step=ckpt.model_checkpoint_path.split('/')[-1].split('_')[-1]
        for i in range(1682):
            start=(i*batch_size)%168200  #traindata.shape[0]
            end=start+batch_size
            batch_x = test[start:end]
            #print("di  "+str(i))
            out=sess.run(out,feed_dict = {x:batch_x})
            out=np.array(out).reshape([100,1])
            pred=np.vstack((pred,out))

pred.to_csv('Result1.csv')

INFO:tensorflow:Restoring parameters from logs/model.ckpt-9647


TypeError: Fetch argument array([[0],
       [0],
       [0],
       [0],
       [2],
       [2],
       [2],
       [0],
       [0],
       [0],
       [2],
       [0],
       [2],
       [2],
       [2],
       [2],
       [0],
       [0],
       [2],
       [0],
       [2],
       [2],
       [2],
       [2],
       [0],
       [0],
       [2],
       [0],
       [2],
       [2],
       [2],
       [2],
       [0],
       [0],
       [2],
       [0],
       [2],
       [8],
       [2],
       [2],
       [0],
       [0],
       [2],
       [0],
       [2],
       [2],
       [8],
       [2],
       [0],
       [0],
       [2],
       [0],
       [2],
       [2],
       [2],
       [8],
       [0],
       [0],
       [2],
       [0],
       [2],
       [2],
       [2],
       [2],
       [0],
       [0],
       [2],
       [0],
       [2],
       [2],
       [2],
       [2],
       [0],
       [0],
       [2],
       [0],
       [8],
       [8],
       [2],
       [2],
       [0],
  